In [1]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import glob
import os
import polars as pl
# import duckdb as dd
from tqdm import tqdm
import matplotlib.pyplot as plt
import cv2
from pydicom import dcmread
import warnings
from sklearn.preprocessing import LabelEncoder
import pickle
import gc
import ctypes
from sklearn.model_selection import train_test_split

import tensorflow as tf
from tensorflow import keras
from tensorflow.python.keras import backend as K

2024-08-19 03:59:36.080694: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-08-19 03:59:36.080821: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-08-19 03:59:36.225387: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


In [2]:
import tensorflow as tf
try: # detect TPUs
    tpu = tf.distribute.cluster_resolver.TPUClusterResolver()
    tf.tpu.experimental.initialize_tpu_system(tpu)
    strategy = tf.distribute.TPUStrategy(tpu)
except ValueError: # detect GPUs
    strategy = tf.distribute.MirroredStrategy() # for GPU or multi-GPU machines
    #strategy = tf.distribute.get_strategy() # default strategy that works on CPU and single GPU
    #strategy = tf.distribute.experimental.MultiWorkerMirroredStrategy() # for clusters of multi-GPU machines

print("Number of accelerators: ", strategy.num_replicas_in_sync)

Number of accelerators:  1


In [2]:
model = keras.models.\
load_model("/kaggle/input/keras_base_lnfn_l1_l2/keras/default/1/keras_base_left_neural_foraminal_narrowing_l1_l2.h5")

In [3]:
submission = pd.read_csv('/kaggle/input/rsna-2024-lumbar-spine-degenerative-classification/sample_submission.csv')
submission['row_id'] = 'samples'

In [18]:
Test = True
config = {}

if Test:
    config['root_file_path'] = '/kaggle/input/rsna-2024-lumbar-spine-degenerative-classification/train_images'
    config['start'] = 85
    config['end'] = 110
    studies = os.listdir(config['root_file_path'])[config['start']:config['end']]
    test_dict = {}
    image_files = []

    for study in studies:
        for dirname, _, filenames in os.walk(config['root_file_path']+'/'+study):
            for filename in filenames:
                test_dict[os.path.join(dirname, filename).split('/')[-3]] = image_files
                image_files.append(os.path.join(dirname, filename))
else:
    config['root_file_path'] = '/kaggle/input/rsna-2024-lumbar-spine-degenerative-classification/test_images/'
    test_dict = {}
    image_files = []

    for dirname, _, filenames in os.walk(config['root_file_path']):
        for filename in filenames:
            test_dict[os.path.join(dirname, filename).split('/')[-3]] = image_files
            image_files.append(os.path.join(dirname, filename))

#### Non-vectorized version, for Medium

In [19]:
list(test_dict.keys())

['52695609',
 '3718047621',
 '2799878304',
 '2090173285',
 '1606523187',
 '3240929780',
 '545140827',
 '1670552101',
 '113121178',
 '3129894645',
 '3293419987',
 '2946997448',
 '3587842489',
 '2538919007',
 '454297464',
 '2379426952',
 '1805845915',
 '228290246',
 '3503499724',
 '934686772',
 '3448721674',
 '3748910433',
 '413910863',
 '3542358517',
 '3635664607']

In [21]:
len(test_dict['2538919007'])

1981

In [22]:
test_dict['2538919007'][0]

'/kaggle/input/rsna-2024-lumbar-spine-degenerative-classification/train_images/52695609/3284178089/12.dcm'

In [23]:
dicom_ds = dcmread(test_dict['2538919007'][0])
img_array = dicom_ds.pixel_array
np.resize(np.mean(img_array.T, axis=0),(128,))

array([  0.      , 110.334375, 148.903125, 185.865625, 190.0125  ,
       189.575   , 187.7875  , 186.065625, 185.915625, 184.29375 ,
       181.1375  , 182.26875 , 183.0375  , 181.9625  , 183.246875,
       183.746875, 183.703125, 183.4     , 182.31875 , 184.16875 ,
       183.590625, 177.50625 , 174.225   , 172.4625  , 167.0625  ,
       162.04375 , 163.98125 , 169.1375  , 172.609375, 173.328125,
       173.675   , 174.15    , 174.18125 , 173.903125, 176.109375,
       181.440625, 188.10625 , 191.465625, 188.38125 , 188.365625,
       192.984375, 190.9375  , 187.571875, 190.896875, 191.978125,
       189.278125, 189.33125 , 193.646875, 196.06875 , 191.96875 ,
       190.609375, 194.21875 , 195.415625, 196.021875, 196.915625,
       195.4875  , 198.74375 , 206.65625 , 208.00625 , 206.5875  ,
       207.25625 , 203.496875, 203.278125, 205.7375  , 203.225   ,
       202.44375 , 202.053125, 201.65    , 206.9     , 207.73125 ,
       206.428125, 212.390625, 216.978125, 215.921875, 213.403

In [24]:
feature_resized = []

for st in tqdm(test_dict.keys()):
    for file in test_dict[st]:
        dicom_ds = dcmread(file)
        img_array = dicom_ds.pixel_array
        feature_resized.append(np.resize(np.mean(img_array.T, axis=0),(128,)))
    final = np.array(feature_resized)
    y_proba_non_vect = model.predict(final)

  0%|          | 0/25 [00:00<?, ?it/s]

62/62 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step


  4%|▍         | 1/25 [00:37<14:58, 37.45s/it]

124/124 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step


  8%|▊         | 2/25 [01:00<11:11, 29.19s/it]

...

 92%|█████████▏| 23/25 [09:11<00:48, 24.12s/it]

1486/1486 ━━━━━━━━━━━━━━━━━━━━ 2s 1ms/step


 96%|█████████▌| 24/25 [09:35<00:24, 24.12s/it]

1548/1548 ━━━━━━━━━━━━━━━━━━━━ 2s 1ms/step


100%|██████████| 25/25 [10:00<00:00, 24.02s/it]


#### Vecotorised version, for Medium

In [25]:
def get_feature_array(img_file_list):
    dicom_ds = dcmread(img_file_list)
    img_array = dicom_ds.pixel_array
    return np.resize(np.mean(img_array.T, axis=0),(128,))

vfunc = np.vectorize(get_feature_array, otypes=[object])

def get_predictions(key, model_to_use):
    if Test:
        pbar.update(1)
    final_feature_list = vfunc(test_dict[key]).tolist()
    final = np.array(final_feature_list)
    return model_to_use.predict(final)

vfunc2 = np.vectorize(get_predictions, otypes=[object])

global pbar 
pbar = 0

with tqdm(total=len(test_dict.keys())) as pbar:
    y_proba = vfunc2(list(test_dict.keys()), model)

  0%|          | 0/25 [00:00<?, ?it/s]

62/62 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step


  8%|▊         | 2/25 [00:21<04:06, 10.71s/it]

62/62 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step


 12%|█▏        | 3/25 [00:43<05:37, 15.32s/it]

...

 96%|█████████▌| 24/25 [08:21<00:21, 21.77s/it]

62/62 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step


100%|██████████| 25/25 [08:43<00:00, 21.92s/it]

62/62 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step


100%|██████████| 25/25 [09:05<00:00, 21.82s/it]


In [24]:
with strategy.scope():
    
    def get_feature_array(img_file_list):
        dicom_ds = dcmread(img_file_list)
        img_array = dicom_ds.pixel_array
        return np.resize(np.mean(img_array.T, axis=0),(128,))

    vfunc = np.vectorize(get_feature_array, otypes=[object])
    
    def get_predictions(key, model_to_use):
        if Test:
            pbar.update(1)
        final_feature_list = vfunc(test_dict[key]).tolist()
        final = np.array(final_feature_list)
        return model_to_use.predict(final)

    vfunc2 = np.vectorize(get_predictions, otypes=[object])
    
    if Test:
        global pbar 
        pbar = 0

        with tqdm(total=len(test_dict.keys())) as pbar:
            y_proba = vfunc2(list(test_dict.keys()), model)
    else:
        y_proba = vfunc2(list(test_dict.keys()), model)

 10%|█         | 1/10 [00:07<01:11,  7.99s/it]


KeyboardInterrupt: 

In [31]:
rows = {}
# weight_dict = {'normal_mild':1, 'moderate':2, 'severe':4}
conditions = ['spinal_canal_stenosis', 'neural_foraminal_narrowing', 'subarticular_stenosis']
sides = ['left', 'right']
vertebrae_levels = ['l1_l2', 'l2_l3', 'l3_l4', 'l4_l5', 'l5_s1']
severity_levels = ['normal_mild', 'moderate', 'severe']

for c in conditions:
    for v in vertebrae_levels:
        if c != 'spinal_canal_stenosis':
            for s in sides:
                for st in test_dict.keys():
                    if s+'_'+c+'_'+v == 'left_neural_foraminal_narrowing_l1_l2':
                        #print("going for model ", st+'_'+s+'_'+c+'_'+v)
                        pass
                    else:
                        #print(st+'_'+s+'_'+c+'_'+v)
                        rows[st+'_'+s+'_'+c+'_'+v] = np.array([0.333333, 0.333333, 0.333333])
        else:
            for st in test_dict.keys():
                #print(st+'_'+c+'_'+v)
                rows[st+'_'+c+'_'+v] = np.array([0.333333, 0.333333, 0.333333])

In [32]:
test_dict_key_list = list(test_dict.keys())

for st in range(len(test_dict_key_list)):
    # print(test_dict_key_list[st])
    rows[test_dict_key_list[st]+'_left_neural_foraminal_narrowing_l1_l2'] = np.mean(y_proba[st], axis=0)

2532413137
4031357862
3731783147
4115208111
3588716120
2608235425
1020394063
3563559233
2563313484
82066307


In [37]:
for row_id, feature in tqdm(rows.items()):
    feature_set_reshaped = feature.reshape(1, -1)
    predictions = np.ascontiguousarray(feature_set_reshaped)
    df = pd.DataFrame(predictions, columns=severity_levels)
    df.insert(loc=0, column='row_id', value=row_id)
    submission = pd.concat([submission,df]).reset_index(drop=True)
    
i = submission[(submission.row_id == 'samples')].index
submission = submission.drop(i).reset_index(drop=True)

100%|██████████| 250/250 [00:00<00:00, 1354.47it/s]


In [39]:
submission.to_csv('submission.csv', index=False)